In [1]:
import numpy as np
np.seterr(all='raise')
import sys
sys.path.append('../')

from meshmaker.base import Base, Laziness
from meshmaker.model import Model
from meshmaker.mesh import Mesh
from meshmaker.pmesh import ParamMesh, MetaMesh, MetaScene
from meshmaker.seam import Seam
from meshmaker.tform import TForm
from meshmaker.vec3 import vec3
from meshmaker.quat import quat
from meshmaker.planargraph import planargraph
#from meshmaker.geometry import batch, slide, loop_offset, loop_normal, loop_contains, isnear, near, loopO
from meshmaker.geometry import batch, slide, isnear, near, loop_normal, bbox
from meshmaker.mgl import show, MainShader, EdgeShader, WireShader
from meshmaker.plt import *
from collections import defaultdict
import json
from functools import partial

show = partial(show, programs=[MainShader(), EdgeShader(), WireShader(color=vec3.U().yz())])

In [5]:
class Arch:
    
    def __init__(self, x, y, z=0, w=0.1, d=0.1, n=8, m=8):
        self.x, self.y, self.z, self.w, self.d, self.n, self.m = x, y, z, w, d, n, m

    def _splines(self, x, y, n, dZ=1):
        xN = loop_normal(x)
        yN = loop_normal(y).fp()
        seams = [Seam(u, v, n, xN, yN).loop() for u, v in zip(y, x)]
        aaa, bbb = bbox([x for y in seams for x in y])
        dz = (bbb.z - aaa.z)
        dZ = vec3(1, 1, dZ / dz)
        seams = [vec3(0, 0, aaa.z).trnps(dZ.sclps(vec3(0, 0, -aaa.z).trnps(s))) for s in seams]
        return seams

    def _feet(self):
        x = vec3.O().ring(1, self.n, False)
        x = vec3(self.w, self.d, 1).sclps(x)
        y = [p.cp() for p in x]
        y = quat.av(np.pi, vec3.Y()).rot(y)
        x = vec3.Z(self.z).trnps(x)
        y = vec3.Z(self.z).trnps(y)
        q = quat.rotz(vec3.X().saxy((self.y - self.x).nrm()))
        x = q.rot(x)
        y = q.rot(y)
        x = self.x.trnps(x)
        y = self.y.trnps(y)
        return x, y

    def __call__(self):
        arch = Mesh()
        
        x, y = self._feet()
        seams = self._splines(x, y, self.m)
        
        seams = [[arch.av(p) for p in s] for s in seams]
        arch.bridges(seams, m=0, n=1)
        
        #uv_seams = [list(slide(s, 2, 1)) for s in seams]
        #uv_seams = set([tuple(x) for y in uv_seams for x in y])
        #mesh.uvs = mesh.vertex_uvs(seams=uv_seams)
        arch.normals = arch.vertex_normals(smooth=True)
        
        return arch

In [6]:
class WallGraph:
    
    def __init__(self):
        self.pg = planargraph()
        
    def aa(self, u, v, **arch):
        u = self.pg.nv(u, column=False)
        v = self.pg.nv(v, column=False)
        return self.pg.ne(u, v, arch=True, **arch)
        
    def aw(self, u, v, **wall):
        u = self.pg.nv(u, column=False)
        v = self.pg.nv(v, column=False)
        return self.pg.ne(u, v, arch=False, **wall)
        
    def ac(self, u, **column):
        return self.pg.nv(u, column=True, **column)
        
    @staticmethod
    def _column(v):
        bottom = v.ring(v.properties.get('r', 0.1),
                        v.properties.get('n', 4),
                        inscribe=False)
        top = vec3.Z(v.properties.get('z', 1)).trnps([p.cp() for p in bottom])
        column = Mesh()
        column.bridge(bottom, top)
        column.normals = column.vertex_normals(smooth=True)
        return TForm.from_meshes(column, texture='generic_11')
        
    @staticmethod
    def _wall(u, v, edge):
        z = edge.get('z', 1)
        dZ = vec3.Z(z)
        dN = vec3.Z().crs((u - v).nrm()).scl(edge.get('w', 0.1))
        left = [u + dN, v + dN, v + dZ + dN, u + dZ + dN]
        right = [p - (dN * 2) for p in left]
        right.reverse()
        wall = Mesh()
        wall.af(left)
        wall.af(right)
        wall.af([left[3], left[2], right[1], right[0]])
        return TForm.from_meshes(wall, texture='generic_12')
        
    def scene(self):
        tf = TForm()
        for i, v in enumerate(self.pg.vertices):
            if v is None or v.properties.get('column', False) is False:
                continue
            column = self._column(v)
            tf.add(column)
        for i, j, edge in self.pg.edges:
            u, v = self.pg.vertices[i].cp(), self.pg.vertices[j].cp()
            isarch = edge.pop('arch')
            if isarch:
                arch = Arch(u, v, **edge)()
                tf.add(TForm.from_meshes(arch, texture='generic_16'))
            else:
                wall = self._wall(u, v, edge)
                tf.add(wall)
        return tf

In [7]:
show(Arch(vec3(-1, 0, 0), vec3(1, 0, 0), 0, 0.2, 0.2))

TypeError: 'Arch' object is not iterable

In [8]:
floor = Mesh()
floor.af(vec3.O().ring(5, 4, False))
scene = TForm.from_meshes(floor)

wg = WallGraph()
for x, y in slide(vec3.O().ring(2, 4), 2):
    #wg.aw(x, y, z=1, d=0.2, w=0.1, n=16)
    wg.aa(x, y, z=1, d=0.2, w=0.2, n=16, m=32)
    wg.ac(x.cp(), z=1, r=0.2, n=16)

scene.add(wg.scene())

show(MetaScene(scene))

Building Shader 1:<meshmaker.mgl.MainShader object at 0x7fb0b57bd9e8>
Building Shader 2:<meshmaker.mgl.EdgeShader object at 0x7fb0bf839208>
Building Shader 3:<meshmaker.mgl.WireShader object at 0x7fb0bf839400>
Loaded texture: generic_11 (../resources/textures/generics/generic_11.png)
Loaded texture: generic_16 (../resources/textures/generics/generic_16.png)
Loaded texture: generic_8 (../resources/textures/generics/generic_8.png)
